In [1]:
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import math
from sklearn.preprocessing import normalize
import jieba
import re
import os
import time

In [ ]:
# 設定 dataframe 顯示內容大小
# 顯示所有列
pd.set_option('display.max_columns', None)
# 顯示所有行
pd.set_option('display.max_rows', None)
# 設置 value 的顯示長度為 100，預設為 50
pd.set_option('max_colwidth',1000)

`將 main 和 content 合併，以計算 tf-idf 與 worcloud 用`

In [ ]:
raw = pd.read_csv('D:/期末專題/AI法官/rawData/forModel.csv', encoding='utf-8')
raw.head(1)

In [ ]:
m = raw.loc[:, 'judge_main']
c = raw.loc[:, 'judge_content']
m_c_lst = []  # main + content

for i in range(0, len(m)):
    m_c[i] = ''.join(m[i]+c[i])
    m_c_lst.append(m_c[i])
# m_c
    

raw.insert(4, 'judge_main_content', m_c_lst)

raw.to_csv('D:/期末專題/AI法官/rawData/forModel_mainContent.csv', encoding='utf-8-sig', index=False)

### 建立各罪 csv

In [ ]:
l = [1, 2, 3, 4]
ll = ['a', 'b', 'c', 'd']

# a = list(zip(l, ll))
# a_dict = dict(a)
for i in l.keys():
    print(i)

In [2]:
raw = pd.read_csv('D:/期末專題/AI法官/rawData/forModel_mainContent.csv', encoding='utf-8')

In [6]:
'''
judge_multicrimeCategory

1 傷害直系血親尊親屬致死
2 傷害直系血親尊親屬
3 傷害致死
4 過失傷害致重傷
5 傷害致重傷
6 重傷害未遂
7 重傷致死
8 重傷害
9 教唆傷害
10 肇事逃逸
11 過失傷害
12 傷害
13 無罪
'''
df_1 = raw[(raw['judge_multiCrimeCategory'] == '傷害直系血親尊親屬致死')]
# print(len(df_1))
df_2 = raw[(raw['judge_multiCrimeCategory'] == '傷害直系血親尊親屬')]
df_3 = raw[(raw['judge_multiCrimeCategory'] == '傷害致死')]
df_4 = raw[(raw['judge_multiCrimeCategory'] == '過失傷害致重傷')]
df_5 = raw[(raw['judge_multiCrimeCategory'] == '傷害致重傷')]
df_6 = raw[(raw['judge_multiCrimeCategory'] == '重傷害未遂')]
df_7 = raw[(raw['judge_multiCrimeCategory'] == '重傷致死')]
df_8 = raw[(raw['judge_multiCrimeCategory'] == '重傷害')]
df_9 = raw[(raw['judge_multiCrimeCategory'] == '教唆傷害')]
df_10 = raw[(raw['judge_multiCrimeCategory'] == '肇事逃逸')]
df_11 = raw[(raw['judge_multiCrimeCategory'] == '過失傷害')]
df_12 = raw[(raw['judge_multiCrimeCategory'] == '傷害')]
df_13 = raw[(raw['judge_multiCrimeCategory'] == '無罪')]

lst = [df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10, df_11, df_12, df_13]
crime = ['傷害直系血親尊親屬致死', 
         '傷害直系血親尊親屬', 
         '傷害致死', 
         '過失傷害致重傷', 
         '傷害致重傷', 
         '重傷害未遂', 
         '重傷致死', 
         '重傷害',
         '教唆傷害',
         '肇事逃逸',
         '過失傷害',
         '傷害',
         '無罪']

for i in range(1, 14):
    lst[i-1].to_csv(f'D:/期末專題/AI法官/rawData/crimeCategory/crimeCategory_{i}_{crime[i-1]}.csv', encoding='utf-8-sig', index=False)

### TF：依各罪比較

* step 1 `讀取 該罪對應內文` 

* step 2 `將 內文+正規+結巴 存成 txt`

* step 3 `計算 TF/IDF/TF-IDF 內文_正規_結巴.txt` 

`依各罪直接存成一個 corpus`

In [2]:
'''
df1 傷害直系血親尊親屬致死
df2 傷害直系血親尊親屬
df3 傷害致死
df4 過失傷害致重傷
df5 傷害致重傷
df6 重傷害未遂
df7 重傷致死
df8 重傷害
df9 教唆傷害
df10 肇事逃逸
df11 過失傷害
df12 傷害
df13 無罪

'''

#lst = [df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10, df_11, df_12, df_13]

crime = ['傷害直系血親尊親屬致死', 
         '傷害直系血親尊親屬', 
         '傷害致死', 
         '過失傷害致重傷', 
         '傷害致重傷', 
         '重傷害未遂', 
         '重傷致死', 
         '重傷害',
         '教唆傷害',
         '肇事逃逸',
         '過失傷害',
         '傷害',
         '無罪']
startTime = time.time()
for i in range(0, len(crime)):
    # step 1 讀取 該罪對應內文
    df = pd.read_csv(f'D:/期末專題/AI法官/rawData/crimeCategory/crimeCategory_{i+1}_{crime[i]}.csv', encoding='utf-8')
    df_content = df.loc[:, 'judge_main_content']

    # step 2 將 內文+正規+結巴 存成 txt
    with open(f'D:/期末專題/AI法官/rawData/crimeCategory/crimeCategory_{i+1}_{crime[i]}.txt', 'w', encoding='utf-8') as f:
        contlst = []
        for content in df_content:
            contlst.append(content)
        
        data_content = ''.join(contlst)
        #print(data_content)
    
        # re
        contlst_re = []
        r4 = "\【.*?】+|\《.*?》+|\#.*?#+|[.!/_,$&%^*()<>+""'?@|:~{}#]+|[——！\，。=？、：“”‘’￥……()《》【】]|[a-zA-Z]+|（|）|，|。|；|○|「|」"
        data_content_ = re.sub(r4,'',data_content)
        contlst_tmp = list(data_content_)
    
        for content in contlst_tmp:
            contlst_re.append(content)
    
        data_content_re = ''.join(contlst_re)

        #print(data_content_re)
    
    
        # jieba
        # load 自訂義 dict
        dict = 'C:/Users/Jeh/word2vec/dict_judge_update_new.txt'
        jieba.load_userdict(dict)
           
        # load 自訂義 stopword
        stopword = 'C:/Users/Jeh/word2vec/stopword_judge_update.txt'
        stopwords = [line.strip() for line in open(stopword, 'r', encoding='utf-8').readlines()]
        data_content_re_lst = str(data_content_re)
        f.write(data_content_re_lst)

    with open(f'D:/期末專題/AI法官/rawData/crimeCategory/crimeCategory_{i+1}_{crime[i]}_jeba.txt', 'w', encoding='utf-8') as new_f:
        with open(f'D:/期末專題/AI法官/rawData/crimeCategory/crimeCategory_{i+1}_{crime[i]}.txt', 'r', encoding='utf-8') as f:            
            for times, data in enumerate(f, 1):
                print(f'開始存取 {crime[i]} 的 jieba.txt')
                data = jieba.cut(data)
                data = [word for word in data if word != ' ']
                data = [word for word in data if word not in stopwords]  # 停用字
                data = ' '.join(data)  # 將 list 轉成 string
                new_f.write(data)

print(f"總花費時間 : {((time.time() - startTime)/60):.2f} 分鐘")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Jeh\AppData\Local\Temp\jieba.cache
Loading model cost 0.659 seconds.
Prefix dict has been built successfully.


開始存取 傷害直系血親尊親屬致死 的 jieba.txt
開始存取 傷害直系血親尊親屬 的 jieba.txt
開始存取 傷害致死 的 jieba.txt
開始存取 過失傷害致重傷 的 jieba.txt
開始存取 傷害致重傷 的 jieba.txt
開始存取 重傷害未遂 的 jieba.txt
開始存取 重傷致死 的 jieba.txt
開始存取 重傷害 的 jieba.txt
開始存取 肇事逃逸 的 jieba.txt
開始存取 過失傷害 的 jieba.txt
開始存取 傷害 的 jieba.txt
開始存取 無罪 的 jieba.txt
總花費時間 : 452.15 分鐘


### `依各罪的各篇判決書內文存成 n個 corpus`

In [ ]:
# 罪類別：其他類 -> 教唆傷害c1 (0筆) + 肇事逃逸c2  (6筆)

In [ ]:
crime = ['傷害直系血親尊親屬致死', 
         '傷害直系血親尊親屬', 
         '傷害致死', 
         '過失傷害致重傷', 
         '傷害致重傷', 
         '重傷害未遂', 
         '重傷致死', 
         '重傷害',
         '教唆傷害',
         '肇事逃逸',
         '過失傷害',
         '傷害',
         '無罪']

startTime = time.time()
for i in range(0, len(crime)):
    folderPath = f'D:/期末專題/AI法官/rawData/crimeCategory/TF-IDF/{i+1}_{crime[i]}'
    if not os.path.exists(folderPath):
        os.makedirs(folderPath)
    folderPath_jieba = f'D:/期末專題/AI法官/rawData/crimeCategory/TF-IDF/{i+1}_{crime[i]}/jieba'
    if not os.path.exists(folderPath_jieba):
        os.makedirs(folderPath_jieba)

    # step 1 讀取 該罪對應內文
    dfn = pd.read_csv(f'D:/期末專題/AI法官/rawData/crimeCategory/crimeCategory_{i+1}_{crime[i]}.csv', encoding='utf-8')
    dfn_content = dfn.loc[:, 'judge_main_content']
# print(len(dfn_content))  #5 63 14 53 34 15 2 303 0 6 44625 18816 2217
        
        
    # step 2 將 內文+正規+結巴 存成 txt
    # 將 每筆內文 存成 txt
    for index in range(0, len(dfn_content)):
        with open(f'{folderPath}/{index+1}_{crime[i]}.txt', 'w', encoding='utf-8') as f1:
            contlst = []
            for content in dfn_content[index]:
                contlst.append(content)
        
            data_content = ''.join(contlst)
#             print(len(data_content)
    
            # re
            contlst_re = []
            r4 = "\【.*?】+|\《.*?》+|\#.*?#+|[.!/_,$&%^*()<>+""'?@|:~{}#]+|[——！\，。=？、：“”‘’￥……()《》【】]|[a-zA-Z]+|（|）|，|。|；|○|「|」"
            data_content_ = re.sub(r4,'',data_content)
            contlst_tmp = list(data_content_)
    
            for content in contlst_tmp:
                contlst_re.append(content)
    
            data_content_re = ''.join(contlst_re)
            #print(data_content_re)
    
            # jieba
            # load 自訂義 dict
            dict = 'C:/Users/Jeh/word2vec/dict_judge_update.txt'
            jieba.load_userdict(dict)
           
            # load 自訂義 stopword
            stopword = 'C:/Users/Jeh/word2vec/stopword_judge_update.txt'
            stopwords = [line.strip() for line in open(stopword, 'r', encoding='utf-8').readlines()]
    
            data_content_re_lst = str(data_content_re)
            f1.write(data_content_re_lst)
    
        with open(f'{folderPath_jieba}/{index+1}_jieba_{crime[i]}.txt', 'w', encoding='utf-8') as new_f:
            with open(f'{folderPath}/{index+1}_{crime[i]}.txt', 'r', encoding='utf-8') as f:
                for times, data in enumerate(f, 1):
                    print(f'開始存取 {crime[i]} 的 jieba.txt')
                    data = jieba.cut(data)
                    data = [word for word in data if word != ' '] 
                    data = [word for word in data if word not in stopwords] # 停用字
                    data = ' '.join(data)  # 將 list 轉成 string
                    new_f.write(data)
print(f"總花費時間 : {((time.time() - startTime)/60):.2f} 分鐘")


In [ ]:
# step 3 計算各類罪的 TF/IDF/TF-IDF 內文_正規_結巴.txt

`TF`

`存取各類罪的各內文檔案`

In [ ]:
import glob
import os


crime = ['1_傷害直系血親尊親屬致死', 
         '2_傷害直系血親尊親屬', 
         '3_傷害致死', 
         '4_過失傷害致重傷', 
         '5_傷害致重傷', 
         '6_重傷害未遂', 
         '7_重傷致死', 
         '8_重傷害',
         '9_教唆傷害',
         '10_肇事逃逸',
         '11_過失傷害',
         '12_傷害',
         '13_無罪']
crimeName = crime[11]
crime_words = []  # 存該罪全部的 word
count = 0  # 記錄各罪的總篇數
for filename in glob.glob(f'D:/期末專題/AI法官/rawData/crimeCategory/TF-IDF/{crimeName}/jieba/*.txt'):
    with open(os.path.join(os.getcwd(), filename), 'r', encoding='utf-8') as f:
        txtn = f.read()
        txtn_s = txtn.split(' ')
        for word in txtn_s:
            crime_words.append(word)  # 存該罪全部的 word
    count+=1

bow_set = set(crime_words)  # 定義詞袋 bag of words
# print(len(bow_set))
wordDict = dict.fromkeys(bow_set, 0)  # 建立詞袋 dict，紀錄每個字的數量
wordDict_record = dict.fromkeys(bow_set, 0)


startTime = time.time()

countN = 1  # 記錄第幾篇
df = pd.DataFrame([wordDict])
for filename in glob.glob(f'D:/期末專題/AI法官/rawData/crimeCategory/TF-IDF/{crimeName}/jieba/*.txt'):
    wordDict = dict.fromkeys(bow_set, 0)  # 建立詞袋 dict，紀錄每個字的數量
    df_sum = pd.DataFrame([wordDict])
    with open(os.path.join(os.getcwd(), filename), 'r', encoding='utf-8') as f:
        txtn = f.read()
        txtn_s = txtn.split(' ')  # 存該罪各篇的 word  ->list
        print('開始存取各篇內文')
        print(f'第 {count} of {countN} 篇')
        for word in txtn_s:
            wordDict[word]+=1
            wordDict_record[word]+=1
        countN+=1
        '''
        tfx,y：frequency of x(word) in y(docs)
        N：total number of y(docs)
        dfx：number of y(docs) contains x(word)
        '''
#     print(len(txtn_s))
#     print(txtn_s)

    df = df.append(wordDict, ignore_index=True)  # 記錄各判決內容的 tf
# print(df)
df_sum = df_sum.append(wordDict_record, ignore_index=True)  # 紀錄全判決內容的 tf
# print(df_sum)
    
df_ = df.drop([0,])  # 刪除 各判決內容的tf 第一個空值 row
df_record = df_sum.drop([0,])  # 刪除 全判決內容的tf 第一個空值 row
# print(df_record)
df_final = df_.append(df_record, ignore_index=True)  # 各判決內容的 tf
# print(df_final)
n_lst = []
for i in range(0, count+1):
    n_lst.append(i)
n_lst.sort(reverse=True)
# print(n_lst)
df_final_T = df_final.reindex(n_lst).T # 全判決內容的tf + 各判決內容的tf
# print(df_final_T)
df_final_T.to_csv(f'D:/期末專題/AI法官/rawData/crimeCategory/TF-IDF/{crimeName}.csv', encoding='utf-8-sig')

print(f"總花費時間 : {((time.time() - startTime)/60):.2f} 分鐘")





'''

# tf-idf
# tfidf = {}
# for word, val in tfBow.items():
#     tfidf[word] = val*idfs[word]
# return tfidf







# tfDict = {}
# bowCount = len(txtn_s)
# for word, count in wordDict.items():
#         tfDict[word] = count/float(bowCount)
#     return tfDict
'''

In [ ]:
# 測試 tf-idf

import glob
import os

crime = ['1_傷害直系血親尊親屬致死', 
         '2_傷害直系血親尊親屬', 
         '3_傷害致死', 
         '4_過失傷害致重傷', 
         '5_傷害致重傷', 
         '6_重傷害未遂', 
         '7_重傷致死', 
         '8_重傷害',
         '9_教唆傷害',
         '10_肇事逃逸',
         '11_過失傷害',
         '12_傷害',
         '13_無罪']
crimeName = crime[0]
crime_words = []  # 存該罪全部的 word
count = 0  # 記錄各罪的篇數
for filename in glob.glob(f'D:/期末專題/AI法官/rawData/crimeCategory/TF-IDF/{crimeName}/jieba/*.txt'):
    with open(os.path.join(os.getcwd(), filename), 'r', encoding='utf-8') as f:
        txtn = f.read()
        txtn_s = txtn.split(' ')
        for word in txtn_s:
            crime_words.append(word)  # 存該罪全部的 word
    count+=1

bow_set = set(crime_words)  # 定義詞袋 bag of words
wordDict = dict.fromkeys(bow_set, 0)  # 建立詞袋 dict，紀錄每個字的數量
# idfDict = dict.fromkeys(bow_set, 0)
wordDict_record = dict.fromkeys(bow_set, 0)

df = pd.DataFrame([wordDict])
for filename in glob.glob(f'D:/期末專題/AI法官/rawData/crimeCategory/TF-IDF/{crimeName}/jieba/*.txt'):
    wordDict = dict.fromkeys(bow_set, 0)  # 建立詞袋 dict，紀錄每個字的數量
    idfDict = dict.fromkeys(bow_set, 0)
    df_sum = pd.DataFrame([wordDict])
    with open(os.path.join(os.getcwd(), filename), 'r', encoding='utf-8') as f:
        txtn = f.read()
        txtn_s = txtn.split(' ')  # 存該罪各篇的 word  ->list
        for word in txtn_s:
            wordDict[word]+=1
            wordDict_record[word]+=1
        
         
        '''
        tfx,y：frequency of x(word) in y(docs)
        N：total number of y(docs)
        dfx：number of y(docs) contains x(word)
        '''
        # tf
          
        tfDict = {}  # -> tf frequency
        idfDict = {}  # -> idf
        N = count  # total number of y(docs)
        bowCount = len(txtn_s)  # total number of y_content
        for word, count_ in wordDict.items():
            print(word)
#             tfDict[word] = count_/float(bowCount)
#         print(tfDict)
        #print("="*200)
        
        # idf
        idfDict = dict.fromkeys(tfDict)
#         print(idfDict)
        
#         for word, val in idfDict.items():
#             if val > 0:
#                 idfDict[word] += 1
#             idfDict[word] = math.log10(N/ float(val)) 
#         print(idfDict)
        
        

#         for doc in wordDict:
# #             print(doc)
#             for word, val in doc.items():
#                 if val > 0:
#                     idfDict[word] += 1
    
        
        # print(len(idfDict))  
        

'''
    #print(len(txtn_s))
    #print(txtn_s)

#     df = df.append(wordDict, ignore_index=True)  # 記錄各判決內容的 tf
#     df_sum = df_sum.append(wordDict_record, ignore_index=True)  # 紀錄全判決內容的 tf
    
# df_ = df.drop([0,])  # 刪除 各判決內容的tf 第一個空值 row
# df_record = df_sum.drop([0,])  # 刪除 全判決內容的tf 第一個空值 row
# df_final = df_.append(df_record, ignore_index=True)  # 各判決內容的 tf
# n_lst = []
# for i in range(1, count+1):
#     n_lst.append(i)
# n_lst.sort(reverse=True)
# df_final_T = df_final.reindex(n_lst).T # 全判決內容的tf + 各判決內容的tf
# print(df_final_T)
#df_final_T.to_csv(f'D:/期末專題/AI法官/rawData/crimeCategory/TF-IDF/{crimeName}.csv', encoding='utf-8-sig')



'''

# tf-idf
# tfidf = {}
# for word, val in tfBow.items():
#     tfidf[word] = val*idfs[word]
# return tfidf


# tfDict = {}
# bowCount = len(txtn_s)
# for word, count in wordDict.items():
#         tfDict[word] = count/float(bowCount)
#     return tfDict


`test re`

In [ ]:
te = '傷害致死，過失傷害致重傷後，重傷致無法行走，導致傷害致重傷'


r4 = "\【.*?】+|\《.*?》+|\#.*?#+|[.!/_,$&%^*()<>+""'?@|:~{}#]+|[——！\，。=？、：“”‘’￥……()《》【】]|[a-zA-Z]+|（|）|，|。|；|○|「|」"
data_content = re.sub(r4,'',te)
# print(data_content)
content1 = list(data_content)

with open('test_re.txt', 'w', encoding='utf-8') as new_f:
    for a in content1:
        content2 = ''.join(a)
        new_f.write(content2)
        
with open('test_re_cut.txt', 'w', encoding='utf-8-sig') as new_f:
     with open('test_re.txt', 'r', encoding='utf-8') as f:
        # load 自訂義 dict
        dict = './test.txt'
        jieba.load_userdict(dict)
           
        # load 自訂義 stopword
        # stopword = './stopword_judge.txt'
        # stopwords = [line.strip() for line in open(stopword, 'r', encoding='utf-8').readlines()]
 
    
        for times, data in enumerate(f, 1):
            print('data num:', times)
            data = jieba.cut(data)
            data = [word for word in data if word != ' ']
            data = ' '.join(data)  # 將 list 轉成 string
#             print(data)
            new_f.write(data)
